In [ ]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import calendar

def predict():
    model = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
    
    predictions = {}
    true_values = []
    predicted_values = []

    latest_year = global_df['Year'].max()
    prediction_year = latest_year + 1

    for month in range(1, 13):
        month_data = global_df[global_df['Month'] == month]
        if month_data.empty:
            continue
        
        X = month_data[['Year']]
        y = month_data['Monthly_Expenditure']
        
        model.fit(X, y)
        
        future_year = pd.DataFrame({'Year': [prediction_year]})
        prediction = model.predict(future_year)[0]
        
        predictions[month] = prediction
        predicted_values.append(prediction)
        
        if latest_year in month_data['Year'].values:
            true_value = month_data[month_data['Year'] == latest_year]['Monthly_Expenditure'].values[0]
            true_values.append(true_value)

    predictions_df = pd.DataFrame(list(predictions.items()), columns=['Month', 'Predicted_Monthly_Expenditure'])
    predictions_df['Month_Name'] = predictions_df['Month'].apply(lambda x: calendar.month_name[x])
    predictions_df = predictions_df[['Month', 'Month_Name', 'Predicted_Monthly_Expenditure']]

    '''if true_values:
        mae = mean_absolute_error(true_values, predicted_values)
        r2 = r2_score(true_values, predicted_values)
        print(f'Mean Absolute Error: {mae}')
        print(f'R-squared: {r2}')
    else:
        print("Insufficient data for evaluation.")'''
    
    return predictions_df